In [1]:
from multiprocessing import Process, Queue

import math
from math import inf

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

from random import shuffle
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from collections import Counter
from random import random



In [2]:
file_name = "./data-cancer-numerical.csv"
df = pd.read_csv(file_name,delimiter=",")

df.head()


# remove Id collumn :
df = df.drop(['id'],axis=1)
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [3]:
# remove NaaN data
df.isnull().sum()
df.isna().sum()

df = df.dropna(axis=1)
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [4]:
feature_data = df[
    [
        'radius_mean','texture_mean','area_mean','smoothness_mean',
        'compactness_mean','concavity_mean','concave points_mean',
        'symmetry_mean','fractal_dimension_mean',
        'radius_se','texture_se','area_se','smoothness_se',
        'compactness_se','concavity_se','concave points_se',
        'symmetry_se','fractal_dimension_se',
        'radius_worst','texture_worst','area_worst','smoothness_worst',
        'compactness_worst','concavity_worst','concave points_worst',
        'symmetry_worst','fractal_dimension_worst',
        
    ]
]

X = np.asarray(feature_data)
X[0:5]

array([[1.799e+01, 1.038e+01, 1.001e+03, 1.184e-01, 2.776e-01, 3.001e-01,
        1.471e-01, 2.419e-01, 7.871e-02, 1.095e+00, 9.053e-01, 1.534e+02,
        6.399e-03, 4.904e-02, 5.373e-02, 1.587e-02, 3.003e-02, 6.193e-03,
        2.538e+01, 1.733e+01, 2.019e+03, 1.622e-01, 6.656e-01, 7.119e-01,
        2.654e-01, 4.601e-01, 1.189e-01],
       [2.057e+01, 1.777e+01, 1.326e+03, 8.474e-02, 7.864e-02, 8.690e-02,
        7.017e-02, 1.812e-01, 5.667e-02, 5.435e-01, 7.339e-01, 7.408e+01,
        5.225e-03, 1.308e-02, 1.860e-02, 1.340e-02, 1.389e-02, 3.532e-03,
        2.499e+01, 2.341e+01, 1.956e+03, 1.238e-01, 1.866e-01, 2.416e-01,
        1.860e-01, 2.750e-01, 8.902e-02],
       [1.969e+01, 2.125e+01, 1.203e+03, 1.096e-01, 1.599e-01, 1.974e-01,
        1.279e-01, 2.069e-01, 5.999e-02, 7.456e-01, 7.869e-01, 9.403e+01,
        6.150e-03, 4.006e-02, 3.832e-02, 2.058e-02, 2.250e-02, 4.571e-03,
        2.357e+01, 2.553e+01, 1.709e+03, 1.444e-01, 4.245e-01, 4.504e-01,
        2.430e-01, 3.613e-01

In [5]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
    
    def is_leaf(self):
        return self.value is not None

In [6]:
class DecisionTree:
    def __init__(self, max_depth=100, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None

    def _is_finished(self, depth):
        if (depth >= self.max_depth
            or self.n_class_labels == 1
            or self.n_samples < self.min_samples_split):
            return True
        return False
    
    def _entropy(self, y):
        proportions = np.bincount(y) / len(y)
        entropy = -np.sum([p * np.log2(p) for p in proportions if p > 0])
        return entropy

    def _create_split(self, X, thresh):
        left_idx = np.argwhere(X <= thresh).flatten()
        right_idx = np.argwhere(X > thresh).flatten()
        return left_idx, right_idx

    def _information_gain(self, X, y, thresh):
        parent_loss = self._entropy(y)
        left_idx, right_idx = self._create_split(X, thresh)
        n, n_left, n_right = len(y), len(left_idx), len(right_idx)

        if n_left == 0 or n_right == 0: 
            return 0
        
        child_loss = (n_left / n) * self._entropy(y[left_idx]) + (n_right / n) * self._entropy(y[right_idx])
        return parent_loss - child_loss

    def _best_split(self, X, y, features):
        split = {'score':- 1, 'feat': None, 'thresh': None}

        for feat in features:
            X_feat = X[:, feat]
            thresholds = np.unique(X_feat)
            for thresh in thresholds:
                score = self._information_gain(X_feat, y, thresh)

                if score > split['score']:
                    split['score'] = score
                    split['feat'] = feat
                    split['thresh'] = thresh

        return split['feat'], split['thresh']
    
    def _build_tree(self, X, y,):
        depth = self.max_depth
        self.n_samples, self.n_features = X.shape
        self.n_class_labels = len(np.unique(y))

        # stopping criteria
        if self._is_finished(depth):
            most_common_Label = np.argmax(np.bincount(y))
            return Node(value=most_common_Label)

        # get best split
        rnd_feats = np.random.choice(self.n_features, self.n_features, replace=False)
        best_feat, best_thresh = self._best_split(X, y, rnd_feats)

        # grow children recursively
        left_idx, right_idx = self._create_split(X[:, best_feat], best_thresh)
        left_child = self._build_tree(X[left_idx, :], y[left_idx], depth + 1)
        right_child = self._build_tree(X[right_idx, :], y[right_idx], depth + 1)
        return Node(best_feat, best_thresh, left_child, right_child)
    
    def _traverse_tree(self, x, node):
        if node.is_leaf():
            return node.value
        
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

    def fit(self, X, y):
        self.root = self._build_tree(X, y)

    def predict(self, X):
        predictions = [self._traverse_tree(x, self.root) for x in X]
        return np.array(predictions)

In [7]:
#to check how much did algo predict right
def accuracy(y_tes, y_pred):
    correct = 0
    for i in range(len(y_pred)):
        if(y_tes[i] == y_pred[i]):
            correct += 1
    return (correct/len(y_tes))*100

In [8]:
def find_majority(votes):
    vote_count = Counter(votes)
    top_two = vote_count.most_common(2)
    if len(top_two)>1 and top_two[0][1] == top_two[1][1]:
        # It is a tie
        return 0
    return top_two[0][0]

In [9]:
def bootstrap_sample(data, sample_size, with_replacement = True):
    if not with_replacement:
        data = copy(data)

    sample = []
    for _ in range(sample_size):
        if len(data) == 0:
            break

        index = int(random() * len(data))
        sample.append(data[index])

        if not with_replacement:
            data.pop(index)
    
    return np.asarray(sample)

In [27]:
# create N substract trees from the data
# calculate the Dtree for each sub tree
# make vote for each answer
# pick the majority vote each step

# forest_answers = np.array([])

for substractsNum in range(100):
    
    _bs = bootstrap_sample(data=df.dropna(axis=1).values,sample_size=len(df.values))
    _y = _bs[:,[0]][:, 0]
    _X = np.delete(_bs,0,1)
    _X_train, _X_test, _y_train, _y_pred = train_test_split(
        _X, _y, test_size=0.2, random_state=4
    )
    _y_train = _y_train.astype(int)

    _classifier = DecisionTree()

    _classifier.fit(_X_train, _y_train)

    _y_pred = _classifier.predict(_X_test)
    acc = accuracy(_y_train, _y_pred)
    print(acc,_y_pred)


15.604395604395604 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]
16.263736263736263 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]
13.846153846153847 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]
14.065934065934066 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


In [11]:
from sklearn.ensemble import RandomForestClassifier

In [15]:
y = np.asarray(df['diagnosis'])

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)

print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

clf = RandomForestClassifier(max_depth=20, random_state=0)

Train set: (455, 27) (455,)
Test set: (114, 27) (114,)


In [16]:
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=20, random_state=0)

In [20]:
Y_pred = clf.predict(X_test)